In [1]:
# using Singular
# using AbstractAlgebra
using Random
using Symbolics
using LinearAlgebra
using SparseArrays, StaticPolynomials
using SymPy

In [3]:
using Oscar

 -----    -----    -----      -      -----   
|     |  |     |  |     |    | |    |     |  
|     |  |        |         |   |   |     |  
|     |   -----   |        |     |  |-----   
|     |        |  |        |-----|  |   |    
|     |  |     |  |     |  |     |  |    |   
 -----    -----    -----   -     -  -     -  

...combining (and extending) ANTIC, GAP, Polymake and Singular
Version 0.11.3 ... 
 ... which comes with absolutely no warranty whatsoever
Type: '?Oscar' for more information
(c) 2019-2023 by The OSCAR Development Team


In [4]:
?MatrixSpace

search: MatrixSpace basis_matrix_of_rational_span gram_matrix_of_rational_span



```
MatrixSpace(R::NCRing, r::Int, c::Int; cached::Bool = true)
```

Return parent object corresponding to the space of $r\times c$ matrices over the ring $R$. If `cached == true` (the default), the returned parent object is cached so that it can returned by future calls to the constructor with the same dimensions and base ring.


In [9]:
coefficients = ResidueRing(ZZ, 6)
coefficients(9)

3

In [6]:
function exp_vector_building(n,q)#create a vector of length n and max degree is q
    vectors=[0 for i in 1:n]#create a vector of zeros
    for i in 1:q#max degree is q; therefore loop through q times
        vectors[rand(1:n)]+=1
    end
    return vectors
end
exp_vector_building(3,5)

3-element Vector{Int64}:
 3
 0
 2

In [13]:
n = 3
max_degree = 4
R, xs = PolynomialRing(coefficients, ["x$i" for i in 1:n],
                       ordering = :degrevlex)

(Multivariate Polynomial Ring in x1, x2, x3 over Integers modulo 6, nmod_mpoly[x1, x2, x3])

In [31]:
I=MatrixSpace(R,3,3)
A=I(1)
typeof(A)
A[1,1]=5
inv(A)

LoadError: Not implemented

In [33]:
R, x = PolynomialRing(QQ, "x")
K, a = NumberField(x^3 + 3x + 1, "a")
S = MatrixSpace(K, 3, 3)
typeof(S)

AbstractAlgebra.Generic.MatSpace{nf_elem}

In [15]:
C=MPolyBuildCtx(R)
exp_arr=exp_vector_building(n,max_degree)
push_term!(C, coefficients(7),exp_arr)
# push_term!(C, ZZ(1),exp_vector_building(n,max_degree))
f = finish(C)

f

x1*x2*x3^2

In [17]:
function sparse_polynomial(n,t,b,q)
    coefficients = ResidueRing(ZZ, q)#integer modulo q
    
    R,x=PolynomialRing(coefficients, ["x$i" for i in 1:n],
                       ordering = :degrevlex)
    C=MPolyBuildCtx(R)
    for i in 1:t
        push_term!(C,coefficients(rand(1:100)),exp_vector_building(n,q))
    end
    f=finish(C)
    while f==0
        f=sparse_polynomial(n,t,b,q)
#         return f
    end
    return f
end
a=sparse_polynomial(4,4,5,5)
# typeof(a)

3*x1^2*x2*x3*x4 + x1*x2*x3^2*x4 + 2*x1^2*x3*x4^2

In [23]:
I[1,1]=sparse_polynomial(3,4,5,5)

LoadError: MethodError: no method matching setindex!(::AbstractAlgebra.Generic.MatSpace{nmod_mpoly}, ::nmod_mpoly, ::Int64, ::Int64)

# linear algebra

In [18]:
function identity_mat(k,n,B,q)#generate a kbyk identity matrix
    coefficients = ResidueRing(ZZ, q)#integer modulo q
    R,x=PolynomialRing(coefficients, ["x$i" for i in 1:n],
                       ordering = :degrevlex,degree_bound=B)
    S=zeros(R,k,k)
    C=MPolyBuildCtx(R)
    push_term!(C, coefficients(q+1),exp_vector_building(n,0))
    f1=finish(C)
    
    for i in 1:k
        S[i,i]=f1
    end
    return S
end
S=identity_mat(3,4,5,4)


LoadError: MethodError: no method matching PolynomialRing(::Nemo.NmodRing, ::Vector{String}; ordering=:degrevlex, degree_bound=5)
[0mClosest candidates are:
[0m  PolynomialRing(::Nemo.NmodRing, ::Vector{String}; cached, ordering) at /Users/jchen056/.julia/packages/Nemo/g02iz/src/flint/nmod_mpoly.jl:1217[91m got unsupported keyword argument "degree_bound"[39m
[0m  PolynomialRing(::AbstractAlgebra.Ring, ::Vector{String}; cached, ordering) at /Users/jchen056/.julia/packages/AbstractAlgebra/FcxWh/src/MPoly.jl:1297[91m got unsupported keyword argument "degree_bound"[39m
[0m  PolynomialRing(::AbstractAlgebra.Ring, [91m::Pair{var"#s267", var"#s266"} where {var"#s267"<:Union{String, Symbol}, var"#s266"}[39m, [91m::Any...[39m; cached, ordering) at /Users/jchen056/.julia/packages/Oscar/RlsvY/src/Rings/mpoly.jl:23[91m got unsupported keyword argument "degree_bound"[39m
[0m  ...

In [6]:
function generate_U(k,n,t,B,q)
    I_matrix=identity_mat(k,n,B,q)
    for i in 1:Int((k*k-k)/2)
        b=rand(2:k)
        a=rand(1:(b-1))
        I_matrix[a,b]=sparse_polynomial(n,t,B,q)
    end
    return I_matrix
end
U1=generate_U(4,2,2,3,6)
# U2=generate_U(4,2,2,3,6)
# println(U1)
# println(U2)


// ***dError: Equal monomials in p_Merge_q
// ***dError: Equal monomials in p_Merge_q

4×4 Matrix{spoly{n_Zn}}:
 1  3*x1^5*x2 + 3*x1^4*x2^2  3*x1^2*x2^4            5*x1^3*x2^3 + 2*x1*x2^5
 0  1                        2*x1^5*x2 + x1^3*x2^3  0
 0  0                        1                      0
 0  0                        0                      1

In [37]:
function generate_L(k,n,t,B,q)
    I_matrix=identity_mat(k,n,B,q)#generate a k by k identity matrix
    
    for i in 1:Int((k*k-k)/2)
        a=rand(2:k)
        b=rand(1:a-1)
        I_matrix[a,b]=sparse_polynomial(n,t,B,q)
    end
    return I_matrix
end

L=generate_L(4,3,2,0,6)

4×4 Matrix{spoly{n_Zn}}:
 1               0                              0  0
 x1^2*x2^3*x3    1                              0  0
 5*x1^3*x2^2*x3  2*x1^2*x2^3*x3 + x1*x2^3*x3^2  1  0
 0               0                              0  1

In [41]:
# inv(L)
# S=zeros(Sym,5,4)
S=[5 6 6 8; 2 2 2 8; 6 6 2 8; 2 3 6 7]
typeof(S)
I=identity_matrix(R, 3)
typeof(I)

smatrix{spoly{n_Zn}}

In [27]:
function cols_to_keep(k,l)#select l elements from a list of k elements; k choose l
    col_opts=[i for i in 1:k]#a list with all possible column indexes
    for i in 1:(k-l)#let us remove k-l elements from the list
        random_number=size(col_opts,1)
        splice!(col_opts, rand(1:random_number))
    end
    return col_opts
end

cols_to_keep(5,3)

3-element Vector{Int64}:
 2
 4
 5

In [28]:
function random_del_cols(k,l,matrix,n,B,q)
    #     col_opts=[i for i in 1:l]
    col_opts=cols_to_keep(k,l)
    
    coefficients = ResidueRing(ZZ, q)#integer modulo q
    R,x=PolynomialRing(coefficients, ["x$i" for i in 1:n],
                       ordering = :degrevlex,degree_bound=B)
    matrix_kl=zeros(R,k,l)

    
    j=1
    for i in col_opts#columns to keep from the original matrix
        matrix_kl[:,j]=matrix[:,i]
        j+=1
    end
    return col_opts,matrix_kl
end

random_del_cols(4,2,L,3,3,6)

([1, 2], spoly{n_Zn}[1 0; 4*x1^2*x2^4 1; 5*x1^4*x2^2 + 3*x1^3*x2^3 2*x1^5*x2 + 3*x1^3*x2^3; 2*x1^5*x2 + 3*x1^3*x2^3 0])

In [34]:
function find_kl_inverse(to_keep,S_inv,n,B,q)
    k=size(S_inv,1)#find the number of columns
    
    coefficients = ResidueRing(ZZ, q)#integer modulo q
    R,x=PolynomialRing(coefficients, ["x$i" for i in 1:n],
                       ordering = :degrevlex,degree_bound=B)
    ml_inv=zeros(R, length(to_keep),k)
    
    j=1
    for i in to_keep
        ml_inv[j,:]=S_inv[i,:]
        j+=1
    end
    return ml_inv
end
find_kl_inverse([2, 3],L,3,3,6)

2×4 Matrix{spoly{n_Zn}}:
 4*x1^3*x2*x3^2 + 3*x1^2*x2^2*x3^2  1             0  0
 5*x1*x2^2*x3^3                     2*x1*x2*x3^4  1  0